In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('Telcom Customer Churn.csv')


In [2]:
df.drop('TotalCharges', axis =1,inplace=True)


In [3]:
df.drop('customerID', axis =1,inplace=True)

df.replace('', np.nan, inplace = True)

df.dropna(inplace= True)
df.columns


Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'Churn'],
      dtype='object')

In [4]:
df['gender']=df['gender'].fillna(df['gender'].mode()[0])
df['Churn']=df['Churn'].fillna(df['Churn'].mode()[0])

In [5]:
df['gender'] = df['gender'].map({'Female':0, 'Male':1}).astype(int)
df['Partner'] = df['Partner'].map({'Yes':0, 'No':1}).astype(int)
df['Dependents'] = df['Dependents'].map({'Yes':0, 'No':1}).astype(int)
df['PhoneService'] = df['PhoneService'].map({'Yes':0, 'No':1}).astype(int)
df['MultipleLines'] = df['MultipleLines'].map({'No phone service':2,'Yes':0, 'No':1}).astype(int)
df['InternetService'] = df['InternetService'].map({'DSL':0, 'Fiber optic':1, 'No':2}).astype(int)
df['OnlineSecurity'] = df['OnlineSecurity'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
df['OnlineBackup'] = df['OnlineBackup'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
df['DeviceProtection'] = df['DeviceProtection'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
df['TechSupport'] = df['TechSupport'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
df['StreamingTV'] = df['StreamingTV'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
df['StreamingMovies'] = df['StreamingMovies'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
df['Contract'] = df['Contract'].map({'Month-to-month':0, 'One year':1, 'Two year':2}).astype(int)
df['PaperlessBilling'] = df['PaperlessBilling'].map({'Yes':0, 'No':1}).astype(int)
df['PaymentMethod'] = df['PaymentMethod'].map({'Electronic check':0, 'Mailed check':1, 'Bank transfer (automatic)':2,
       'Credit card (automatic)':3}).astype(int)
df['Churn'] = df['Churn'].map({'Yes':1, 'No':0}).astype(int)


In [6]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,0,0,0,1,1,1,2,0,1,0,1,1,1,1,0,0,0,29.85,0
1,1,0,1,1,34,0,1,0,0,1,0,1,1,1,1,1,1,56.95,0
2,1,0,1,1,2,0,1,0,0,0,1,1,1,1,0,0,1,53.85,1
3,1,0,1,1,45,1,2,0,0,1,0,0,1,1,1,1,2,42.30,0
4,0,0,1,1,2,0,1,1,1,1,1,1,1,1,0,0,0,70.70,1


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

y = df['Churn']
X = df.drop('Churn', axis =1)


x_train_orginal, x_test_orginal, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train_orginal)
x_test = scaler.fit_transform(x_test_orginal)

In [8]:
df.dtypes

gender                int32
SeniorCitizen         int64
Partner               int32
Dependents            int32
tenure                int64
PhoneService          int32
MultipleLines         int32
InternetService       int32
OnlineSecurity        int32
OnlineBackup          int32
DeviceProtection      int32
TechSupport           int32
StreamingTV           int32
StreamingMovies       int32
Contract              int32
PaperlessBilling      int32
PaymentMethod         int32
MonthlyCharges      float64
Churn                 int32
dtype: object

In [9]:
from sklearn.svm import SVC

svc_linear = SVC(kernel = 'linear', C = 10, gamma = 0.1)
svc_linear.fit(x_train, y_train)

confidence = svc_linear.score(x_test,y_test)


print(confidence)

0.8246983676366217


In [10]:
from sklearn.linear_model import LogisticRegression


model = LogisticRegression(max_iter=300,n_jobs=-1) 

#TRAINING THE MODEL
model.fit(x_train,y_train)

#TESTING THE MODEL
confidence = model.score(x_test,y_test)


print(confidence)


0.8190205819730305


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid = {'C': [10,100,1000],
              'gamma': [0.1, 1, 10]}



grid_search = GridSearchCV(SVC(kernel = 'rbf'), param_grid, cv=3, return_train_score=True, n_jobs = -1)
grid_search.fit(x_train, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))


Best parameters: {'C': 10, 'gamma': 0.1}
Best cross-validation score: 0.80


In [12]:
import pickle

with open('./churn_prediction2.pkl', 'wb') as f:
    pickle.dump(svc_linear, f)

In [13]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'Churn'],
      dtype='object')

In [14]:
df1 = pd.read_csv('Telcom Customer Churn.csv')

df1['MultipleLines'].unique()

array(['No phone service', 'No', 'Yes'], dtype=object)

In [15]:
df1['SeniorCitizen'].unique()

array([0, 1], dtype=int64)

In [16]:
with open('./churn_prediction2.pkl', 'rb') as f:
    svc_linear_loaded = pickle.load(f)

In [17]:
import pickle
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler




import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
# Creating tkinter window
window = tk.Tk()

width= window.winfo_screenwidth() 
height= window.winfo_screenheight()
#setting tkinter window size
window.geometry("%dx%d" % (width, height))
window.title('Churn_prediction')



#Column 1 
gender = ttk.Label(window, text = "gender :", font = ("Times New Roman", 10)).grid(column = 0,  row = 0, padx = 10, pady = 25)
gender_var=tk.StringVar()
gender_choosen = ttk.Combobox(window, width = 27, textvariable =gender_var)
gender_choosen['values'] = ('Female', 'Male')
gender_choosen.grid(row=0,column=1)
gender_choosen.current(1)

#Column 2
SeniorCitizen = ttk.Label(window, text = "SeniorCitizen :", font = ("Times New Roman", 10)).grid(column = 0,  row = 1, padx = 10, pady = 25)
SeniorCitizen_var = tk.StringVar()
SeniorCitizen_choosen = ttk.Combobox(window, width = 27, textvariable = SeniorCitizen_var)
SeniorCitizen_choosen['values'] = ('yes','no')
SeniorCitizen_choosen.grid(column = 1, row = 1)
SeniorCitizen_choosen.current(1) 


#Column 3
Partner = ttk.Label(window, text = "Partner :", font = ("Times New Roman", 10)).grid(column = 0,  row = 2, padx = 10, pady = 25)
Partner_var = tk.StringVar()
Partner_choosen = ttk.Combobox(window, width = 27, textvariable = Partner_var)
Partner_choosen['values'] = ('Yes', 'No')
Partner_choosen.grid(column = 1, row = 2)
Partner_choosen.current(1)



#Column 4
Dependents = ttk.Label(window, text = "Dependents :", font = ("Times New Roman", 10)).grid(column = 0,  row = 3, padx = 10, pady = 25)
Dependents_var = tk.StringVar()
Dependents_choosen = ttk.Combobox(window, width = 27, textvariable = Dependents_var)
Dependents_choosen['values'] = ('Yes', 'No')
Dependents_choosen.grid(column = 1, row = 3)
Dependents_choosen.current(1)


#Column 5
PhoneService = ttk.Label(window, text = "PhoneService :", font = ("Times New Roman", 10)).grid(column = 0,  row = 4, padx = 10, pady = 25)
PhoneService_var = tk.StringVar()
PhoneService_choosen = ttk.Combobox(window, width = 27, textvariable = PhoneService_var)
PhoneService_choosen['values'] = ('Yes', 'No')
PhoneService_choosen.grid(column = 1, row = 4)
PhoneService_choosen.current(1)


#Column 6
MultipleLines = ttk.Label(window, text = "MultipleLines :", font = ("Times New Roman", 10)).grid(column = 0,  row = 5, padx = 10, pady = 25)
MultipleLines_var = tk.StringVar()
MultipleLines_choosen = ttk.Combobox(window, width = 27, textvariable = MultipleLines_var)
MultipleLines_choosen['values'] = ('No phone service', 'No', 'Yes')
MultipleLines_choosen.grid(column = 1, row = 5)
MultipleLines_choosen.current(1)


#Column 7
InternetService = ttk.Label(window, text = "InternetService :", font = ("Times New Roman", 10)).grid(column = 0,  row = 6, padx = 10, pady = 25)
InternetService_var = tk.StringVar()
InternetService_choosen = ttk.Combobox(window, width = 27, textvariable = InternetService_var)
InternetService_choosen['values'] = ('DSL', 'Fiber optic', 'No')
InternetService_choosen.grid(column = 1, row = 6)
InternetService_choosen.current(1)

#Column 8
OnlineSecurity = ttk.Label(window, text = "OnlineSecurity :", font = ("Times New Roman", 10)).grid(column = 0,  row = 7, padx = 10, pady = 25)
OnlineSecurity_var = tk.StringVar()
OnlineSecurity_choosen = ttk.Combobox(window, width = 27, textvariable = OnlineSecurity_var)
OnlineSecurity_choosen['values'] = ('No', 'Yes', 'No internet service')
OnlineSecurity_choosen.grid(column = 1, row = 7)
OnlineSecurity_choosen.current(1)

#Column 9
OnlineBackup = ttk.Label(window, text = "OnlineBackup :", font = ("Times New Roman", 10)).grid(column = 0,  row = 8, padx = 10, pady = 25)
OnlineBackup_var = tk.StringVar()
OnlineBackup_choosen = ttk.Combobox(window, width = 27, textvariable = OnlineBackup_var)
OnlineBackup_choosen['values'] = ('Yes', 'No', 'No internet service')
OnlineBackup_choosen.grid(column = 1, row = 8)
OnlineBackup_choosen.current(1)

#Column 10
DeviceProtection = ttk.Label(window, text = "DeviceProtection :", font = ("Times New Roman", 10)).grid(column = 0,  row = 9, padx = 10, pady = 25)
DeviceProtection_var = tk.StringVar()
DeviceProtection_choosen = ttk.Combobox(window, width = 27, textvariable = DeviceProtection_var)
DeviceProtection_choosen['values'] = ('Yes', 'No', 'No internet service')
DeviceProtection_choosen.grid(column = 1, row = 9)
DeviceProtection_choosen.current(1)


#Column 11
TechSupport = ttk.Label(window, text = "TechSupport :", font = ("Times New Roman", 10)).grid(column = 10,  row = 0, padx = 10, pady = 25)
TechSupport_var = tk.StringVar()
TechSupport_choosen = ttk.Combobox(window, width = 27, textvariable = TechSupport_var)
TechSupport_choosen['values'] = ('Yes', 'No', 'No internet service')
TechSupport_choosen.grid(column = 11, row = 0)
TechSupport_choosen.current(1)


#Column 12
StreamingTV = ttk.Label(window, text = "StreamingTV :", font = ("Times New Roman", 10)).grid(column = 10,  row = 1, padx = 10, pady = 25)
StreamingTV_var=tk.StringVar()
StreamingTV_choosen = ttk.Combobox(window, width = 27, textvariable = StreamingTV_var)
StreamingTV_choosen['values'] = ('Yes', 'No', 'No internet service')
StreamingTV_choosen.grid(column = 11, row = 1)
StreamingTV_choosen.current(1)


#Column 13
StreamingMovies = ttk.Label(window, text = "StreamingMovies :", font = ("Times New Roman", 10)).grid(column = 10,  row = 2, padx = 10, pady = 25)
StreamingMovies_var=tk.StringVar()
StreamingMovies_choosen = ttk.Combobox(window, width = 27, textvariable = StreamingMovies_var)
StreamingMovies_choosen['values'] = ('Yes', 'No', 'No internet service')
StreamingMovies_choosen.grid(column = 11, row = 2)
StreamingMovies_choosen.current(1)


#Column 14
Contract = ttk.Label(window, text = "Contract :", font = ("Times New Roman", 10)).grid(column = 10,  row = 3, padx = 10, pady = 25)
Contract_var=tk.StringVar()
Contract_choosen = ttk.Combobox(window, width = 27, textvariable = Contract_var)
Contract_choosen['values'] = ('Month-to-month', 'One year', 'Two year')
Contract_choosen.grid(row=3,column=11)
Contract_choosen.current(1)

#Column 15
PaperlessBilling = ttk.Label(window, text = "PaperlessBilling :", font = ("Times New Roman", 10)).grid(column = 10,  row = 4, padx = 10, pady = 25)
PaperlessBilling_var=tk.StringVar()
PaperlessBilling_choosen = ttk.Combobox(window, width = 27, textvariable =PaperlessBilling_var)
PaperlessBilling_choosen['values'] = ('Yes', 'No')
PaperlessBilling_choosen.grid(column = 11, row = 4)
PaperlessBilling_choosen.current(1)


#Column 16
PaymentMethod = ttk.Label(window, text = "PaymentMethod :", font = ("Times New Roman", 10)).grid(column = 10,  row = 5, padx = 10, pady = 25)
PaymentMethod_var=tk.StringVar()
PaymentMethod_choosen = ttk.Combobox(window, width = 27, textvariable = PaymentMethod_var)
PaymentMethod_choosen['values'] = ('Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)')
PaymentMethod_choosen.grid(column = 11, row = 5)
PaymentMethod_choosen.current(1)

#Column 17

MonthlyCharges = ttk.Label(window, text = "MonthlyCharges :", font = ("Times New Roman", 10)).grid(column = 10,  row = 6, padx = 10, pady = 25)
MonthlyCharges_var=tk.StringVar()
MonthlyCharges_entrybox=ttk.Entry(window,width=27,textvariable=MonthlyCharges_var)
MonthlyCharges_entrybox.grid(row=6,column=11)

tenure = ttk.Label(window, text = "tenure :", font = ("Times New Roman", 10)).grid(column = 10,  row = 7, padx = 10, pady = 25)
tenure_var=tk.StringVar()
tenure_entrybox=ttk.Entry(window,width=27,textvariable=tenure_var)
tenure_entrybox.grid(row=7,column=11)


import pandas as pd
DF = pd.DataFrame()

global DB
import pandas as pd
DF = pd.DataFrame(columns=['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges'])

def action():   
    G= gender_var.get()
    DF.loc[0,'gender'] = G
    S = SeniorCitizen_var.get()
    DF.loc[0,'SeniorCitizen']= S
    P= Partner_var.get()
    DF.loc[0,'Partner']=P
    D=Dependents_var.get()
    DF.loc[0,'Dependents']=D
    t=tenure_var.get()
    DF.loc[0,'tenure']=t
    Phone=PhoneService_var.get()
    DF.loc[0,'PhoneService']=Phone
    M =MultipleLines_var.get()
    DF.loc[0,'MultipleLines']=M
    IS= InternetService_var.get()
    DF.loc[0,'InternetService']=IS
    
    O=OnlineSecurity_var.get()
    DF.loc[0,'OnlineSecurity']=O
    
    Online =OnlineBackup_var.get()
    DF.loc[0,'OnlineBackup']=Online
    
    Device=DeviceProtection_var.get()
    DF.loc[0,'DeviceProtection']=Device
    
    Tech =TechSupport_var.get()
    DF.loc[0,'TechSupport']=Tech
    TV=StreamingTV_var.get()
    DF.loc[0,'StreamingTV']= TV
    Movies=StreamingMovies_var.get()
    DF.loc[0,'StreamingMovies']= Movies
    Cont=Contract_var.get()
    DF.loc[0,'Contract']=Cont
    EMP_VAR=PaperlessBilling_var.get()
    DF.loc[0,'PaperlessBilling']=EMP_VAR
    
    CON_PRICE= PaymentMethod_var.get()
    DF.loc[0,'PaymentMethod']=CON_PRICE
    CONS=MonthlyCharges_var.get()
    DF.loc[0,'MonthlyCharges']=CONS
    
    
DB=DF



def Output():
    action()
    
    DB["tenure"] = pd.to_numeric(DB["tenure"])
    DB['gender'] = DB['gender'].map({'Female':0, 'Male':1}).astype(int)
    DB['Partner'] = DB['Partner'].map({'Yes':0, 'No':1}).astype(int)
    DB['Dependents'] = DB['Dependents'].map({'Yes':0, 'No':1}).astype(int)
    DB['PhoneService'] = DB['PhoneService'].map({'Yes':0, 'No':1}).astype(int)
    DB['MultipleLines'] = DB['MultipleLines'].map({'No phone service':2,'Yes':0, 'No':1}).astype(int)
    DB['InternetService'] = DB['InternetService'].map({'DSL':0, 'Fiber optic':1, 'No':2}).astype(int)
    DB['OnlineSecurity'] = DB['OnlineSecurity'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
    DB['OnlineBackup'] = DB['OnlineBackup'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
    DB['DeviceProtection'] = DB['DeviceProtection'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
    DB['TechSupport'] = DB['TechSupport'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
    DB['StreamingTV'] = DB['StreamingTV'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
    DB['StreamingMovies'] = DB['StreamingMovies'].map({'Yes':0, 'No':1,'No internet service':2}).astype(int)
    DB['Contract'] = DB['Contract'].map({'Month-to-month':0, 'One year':1, 'Two year':2}).astype(int)
    DB['PaperlessBilling'] = DB['PaperlessBilling'].map({'Yes':0, 'No':1}).astype(int)
    DB['PaymentMethod'] = DB['PaymentMethod'].map({'Electronic check':0, 'Mailed check':1, 'Bank transfer (automatic)':2,
           'Credit card (automatic)':3}).astype(int)
    DB["MonthlyCharges"] = pd.to_numeric(DB["MonthlyCharges"])
    DB['SeniorCitizen'] = DB['SeniorCitizen'].map({'no':0, 'yes':1}).astype(int)


   
    db = scaler.transform(DB)


    output=svc_linear.predict(db)
    #print(output)
    if output == 1:
        MsgBox = messagebox.showinfo("Churn results", '    Yes      ')
        
    else:
        MsgBox = messagebox.showinfo('Churn results', '     NO      ')
        

def reset():
    MonthlyCharges_var.set("")
    tenure_var.set("")
    
    
 

    
Predict_button=ttk.Button(window,text="Predict",command=Output,width=25)
Predict_button.grid(row=10,column=15)

reset_button=ttk.Button(window,text="Reset",command=reset,width=25)
reset_button.grid(row=10,column=11)


window.mainloop()